# Tests to be implemented in github actions. 

In [1]:
import sys
sys.path.append("../../../xapres/")
import xapres as xa
import numpy
from xapres import load, utils


## Test of file name/number selection in xapres.load_all

In [7]:
# test that we can select and load the same file using two different methods: 1) selecting the file number, 
# and 2) using the filename that we get from 1) to load the same file again.
def test_file_selection():
    directory='gs://ldeo-glaciology/GL_apres_2022/A101'
    fs1 = load.from_dats(max_range=1400)
    fs1.load_all(directory, 
                remote_load = True,
                file_numbers_to_process=[0,1])

    fs2 = load.from_dats(max_range=1400)
    fs2.load_all(directory, 
                remote_load = True,
                file_names_to_process = fs1.dat_filenames_to_process)

    assert fs1.data.identical(fs2.data)

In [8]:
test_file_selection()

## Test the loading of a single dat file from the google bucket.


In [9]:
#import pytest
#from xapres import ApRESDefs

def test_dat_file_loading():
    directory='gs://ldeo-glaciology/GL_apres_2022/A101'
    fs = load.from_dats(max_range=1400)
    fs.load_all(directory, 
                remote_load = True,
                file_numbers_to_process=[0],
                bursts_to_process=[0])
    assert numpy.isclose(fs.data.chirp.mean().values, 0.02611298) 

In [10]:
test_dat_file_loading()

## Test the file search code
Make sure that the 'dat' is not case sensitive and that the search suffix option works as intended

In [15]:
def test_file_search_methods():
    fs = load.from_dats()   
    data_directory = 'data'
    
    higher_level_list_of_dats = fs.list_files(data_directory + "/sample")
    
    # this checks that the list of files is not empty
    assert higher_level_list_of_dats  

    lower_level_list_of_dats = fs.list_files(data_directory + "/sample/polarmetric")
    # test that all the files found in a lower level directory were also found when searching in a higher level directory
    assert all(item in higher_level_list_of_dats for item in lower_level_list_of_dats)

    # test that the case of the extension (DAT vs dat) doesnt matter
    assert len(fs.list_files(data_directory + "/sample/different_case_examples")) == 2

    # test the search_suffix option is working
    assert len(fs.list_files(data_directory + "/sample/polarmetric", search_suffix='HH')) == 1

In [16]:
test_file_search_methods()

AssertionError: 

## Test file selection methods

In [32]:
## Use two different methods for selecting the same ApRES DAT file from a google bucket. 
#  In each case load it and then check that we have loaded the correct file. 

def test_file_selection_methods():
    directory='gs://ldeo-glaciology/GL_apres_2022/A101'
    fs1 = load.from_dats(max_range=1400)
    fs1.load_all(directory, 
                remote_load = True,
                file_numbers_to_process=[0,1])

    fs2 = load.from_dats(max_range=1400)
    fs2.load_all(directory, 
                remote_load = True,
                file_names_to_process = fs1.dat_filenames_to_process)

    assert fs1.data.identical(fs2.data)

In [33]:
test_file_selection_methods()

## Test polarmetric loading

In [40]:
## test polarmetric local loading by loading the same waypoint twice as if it is two different ones and chaeking 
# that you get the same thing twice.

# tests the attended option and the polarmetric option from locally stored dat files (as opposed to cloud stored dat files)

def test_polarmetric_load():
    
    fs = load.from_dats()
    fs.load_all(attended=True, 
                directory=["data/sample/polarmetric", "data/sample/polarmetric"], 
                polarmetric=True)
    
    assert len(fs.data.waypoint) == 2
    assert all(fs.data.isel(waypoint=0).filename.values == fs.data.isel(waypoint=1).filename.values)



In [41]:
test_polarmetric_load()

## Test `generate_xarray` and `load_zarr` wrappers

In [45]:
#  Test `generate_xarray` and `load_zarr` wrappers
def test_wrappers():
    
    from_DAT_unattended = load.generate_xarray(directory='data/sample/single_dat_file/', 
                file_numbers_to_process = [0], 
                bursts_to_process=[0],
                )

    from_zarr = load.load_zarr()



In [46]:
test_wrappers()

In [29]:
# test the displacement calculation

def test_displacement_calculation():
    from_zarr = load.load_zarr()  # lazily load a large APRES dataset from Greenland
    p1 = from_zarr.isel(time=2000).profile_stacked # select a profile 
    p2 = from_zarr.isel(time=2100).profile_stacked # select a different profile 
       
    utils.compute_displacement(p1, p2)    # calculate the displacement between the two profiles

    t = from_zarr.sel(time='2022-07-17').profile_stacked   # select all the profiles on a specfic date
    results = t.displacement_timeseries(bin_size = 30, offset = 3) # compute a time series of displacement from these data. Use non-default values for offset and bin_size 

    assert (abs(results)>0).all().load()

In [30]:
out = test_displacement_calculation()